In [1]:
!pwd

/data


In [4]:
!cd ./Gemini-Hackathon

In [5]:
!pwd

/data


In [ ]:
!conda create -n skyhammer311 python=3.11 -y
!conda activate skyhammer311
!python -m pip install -U pip


In [ ]:
conda create -n skyhammer311 python=3.11 -y
conda activate skyhammer311
python -m pip install -U pip

python -m pip install torch torchvision --index-url https://download.pytorch.org/whl/cu128

python -m pip install unsloth
python -m pip install ipykernel
python -m ipykernel install --user --name skyhammer311 --display-name "skyhammer (py311)"
python -c "from unsloth import FastLanguageModel; print('ok')"
python -m pip install -U transformers peft accelerate bitsandbytes


pip uninstall -y datasets trl unsloth unsloth-zoo
pip install "datasets<4.4.0,>=3.4.1" "trl<=0.24.0,>=0.18.2,!=0.19.0"
pip install unsloth


pip install huggingface_hub wandb
huggingface-cli login
wandb login

In [ ]:
# Just press Enter for defaults, mostly. 
# Key choices: 
# - Multi-GPU: YES
# - Num machines: 1
# - Num processes: 8
# - Mixed precision: bf16 (Since you have L40s)
accelerate config

accelerate launch --multi_gpu --num_processes=8 src/train/train_ddp.py

In [ ]:
# 1. Set Memory Management Flag
export PYTORCH_ALLOC_CONF=expandable_segments:True

# 2. Run on Single GPU (Safe Mode)
CUDA_VISIBLE_DEVICES=0 python src/train/train_ddp.py

In [ ]:
export PYTORCH_ALLOC_CONF=expandable_segments:True
accelerate launch --multi_gpu --num_processes=8 src/train/train_ddp.py